# Data Preparation


In [3]:
import numpy as np
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from zipfile import ZipFile

## Obtaining data
Data is published at https://www.citibikenyc.com/system-data  
Monthly archives  
Data for NYC and Jersey City (starting 2015-09)

In [9]:
d = "data/nyc"
download_path = os.path.join(d, 'raw')
parsed = os.path.join(d , "parsed")

In [5]:
soup = BeautifulSoup(open(os.path.join(d, "data.html")), 'html.parser')
for link in soup.findAll('a'):
    url = link.get('href')
    name = url.rsplit('/', 1)[-1]
    path = os.path.join(download_path, name)
    if name.endswith(".zip") and not os.path.exists(path):
        r = requests.get(url, allow_redirects=True)
        open(path, 'wb').write(r.content)

Unzip and concatenate data, calculate count by day.  
Some have "Start Time" and others have "starttime".  
Concatenating DFs creates an unwieldy DF. Parse data month by month and save grouped output.

In [6]:
def parse_tripdata(path):
    zip_file = ZipFile(path)
    for file in zip_file.infolist():
        if "__MACOSX" not in file.filename:
            df = pd.read_csv(zip_file.open(file.filename))
            if "Start Time" in df.columns:
                df.rename(columns = {"Start Time": "starttime"}, inplace=True)
            df["starttime"] = pd.to_datetime(df["starttime"])
            df_date = df[["starttime"]].groupby(by = df["starttime"].dt.date).count().rename(columns={'starttime':'count'}).reset_index()
    return df_date

In [10]:
for path in os.listdir(d):
    if path.endswith(".zip"):
        full_path = os.path.join(d, path)
        out_path = os.path.join(parsed, path + ".csv")
        if not os.path.exists(out_path):
            print(path)
            df_parsed = parse_tripdata(full_path)
            df_parsed.to_csv(os.path.join(parsed, path + ".csv"))

In [22]:
dfs= []

for path in os.listdir(parsed):
    full_path = os.path.join(parsed, path)
    df = pd.read_csv(full_path, index_col=0)
    dfs.append(df)
    
df_nyc = pd.concat(dfs)
df_nyc.rename(columns = {"starttime": "date"}, inplace=True)
df_nyc.sort_values(by="date", inplace=True)
# Group stray values that got into a wrong month
df_nyc = df_nyc.groupby("date").sum().reset_index()
df_nyc["date"] = pd.to_datetime(df_nyc["date"])
df_nyc.head()

,date,count
0,2013-06-01,8722
1,2013-06-02,15971
2,2013-06-03,7598
3,2013-06-04,15782
4,2013-06-05,15690


In [29]:
df_nyc["Year"] = pd.DatetimeIndex(df_nyc["date"]).year
df_nyc["Month"] = pd.DatetimeIndex(df_nyc["date"]).month
df_nyc["Week"] = pd.DatetimeIndex(df_nyc["date"]).week
df_nyc["Day"] = pd.DatetimeIndex(df_nyc["date"]).day
df_nyc["Day of Week"] = pd.DatetimeIndex(df_nyc["date"]).dayofweek
df_nyc["Day of Year"] = pd.DatetimeIndex(df_nyc["date"]).dayofyear

/Users/svrogozhin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
df_nyc.head()

,date,count,Year,Month,Week,Day,Day of Week,Day of Year
0,2013-06-01,8722,2013,6,22,1,5,152
1,2013-06-02,15971,2013,6,22,2,6,153
2,2013-06-03,7598,2013,6,23,3,0,154
3,2013-06-04,15782,2013,6,23,4,1,155
4,2013-06-05,15690,2013,6,23,5,2,156


In [31]:
df_nyc.to_csv(os.path.join(d, "nyc-daily.csv"))